In [1]:
import codecs
import re
import numpy as np
import os
from sklearn import tree
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [2]:
## 讀WordProcss.py斷完的結果 ##
cwd = os.getcwd()

seg_news = list()
category = list()

with codecs.open(cwd+'/data/seg.csv','r',encoding='utf-8') as seg_file:
    for seg in seg_file.readlines():
        line = seg.split(',')
        seg_news.append(line[0].strip())
        category.append(line[1].strip())

print(seg_news[109])
print(category[109])

旬 老人 孙女 参加 新 农合 无法 报销 药费 华商 报 安康 讯 记者 赵伯平 上 小学 孙女 学校 参加 城镇 居民 医疗 保险 参加 农村 合作医疗 安康市 汉滨区 恒口镇 和平村 老人 胡先元 患病 住院 费用 无法 报销 家人 跑 部门 问题 无法 解决 出院 医疗费 报销 了 家 人 上 小学 女儿 参加 农村 合作医疗 胡先元 儿子 胡登军 说 上 小学 女儿 学校 带 回 通知 称 在校生 参加 城镇 居民 医疗 保险 家里人 要求 交 钱 孩子 办 保险 父亲 胡先元 肠胃 穿 孔 住 进 汉滨区 医院 治疗费 花 出院 家里人 带 手续 去 医院 报销 结果 医院 说 新农合 户口本 人数 参保 如 人 参保 能 报销 医疗费 胡登军 家 记者 恒口 小学 下发 给 家长 告知书 看到 汉滨区 政府 文件 精神 在校 参加 城镇 居民 保险 参加 农 合 胡登军 说 女儿 学校 要求 购买 保险 想到 父亲 医疗费 无法 报销 孙女 参加 城镇 居民 医保 恒口 小学 校长 熊兰功 介绍 甲流 暴发 时 汉滨区 要求 全区 在校生 办理 城镇 居民 医疗 保险 确保 学生 患病 能 得到 保障 全校 多 学生 办理 城镇 居民 医疗 保险 汉滨区 医院 院长 马明宝 说 国家 政策 规定 参加 农村 合作医疗 户 为 单位 参保 人 参保 规定 能 报销 区 合疗办 反映 希望 区 有关 部门 能 牵头 出面 解决 恒口镇 合疗办 主任 李贵清 说 镇上 上级 有关 部门 政策 执行 接到 群众 问题 反映 参合 能 报销 医疗费 保证 参合率 农民 参加 新 农 合 参合 能 报销 医疗费 汉滨区 合疗办 主任 汪富强 说 区 文件 规定 新 农 合 坚持 家庭 为 单位 参加 农村 合作医疗 原则 参加 农村 合作医疗 户口本 确定 人数 为 准 参加 家庭 上 户口 人员 应 缴纳 参合 基金 交 清人 均元 参合 基金 收缴 工作 结束 参保 可能 参合 能 报销 医疗 费用 政策 具体 执行者 胡先元 问题 区 无法 解决 汪富强 说 在校生 参加 城镇 居民 医疗 保险 开始 类似 和平村 胡先元 遇到 问题 全区 显现 估计 月 问题 突出 汪富强 表示 区 合疗办 收集 类似 问题 积极 上级 部门 反映 拿 出 办法 来 解决

In [4]:
## 找關鍵字 ##
vectorizer = TfidfVectorizer(max_features=3000) #max_features留下的最大特徵數
tfidf = vectorizer.fit_transform(seg_news)
print(tfidf.shape)

words = vectorizer.get_feature_names()
print(words[:10])

data = list() #留下的詞
labels = list() #類別

## write keyword file ##
with codecs.open(cwd+'/data/keyword.csv','w',encoding='utf-8') as kwd:
    kwd.write('\ufeff')
    
    for i in range(len(seg_news)):
        print('----Document %d----' % (i+1),end='\r')
        
        keywords = list()
        
        for j in range(len(words)):
            if tfidf[i,j] > 1e-2: 
                kwd.write(words[j]+' ')
                keywords.append(words[j])
                
        kwd.write(','+str(category[i])+'\r')
        data.append(' '.join(keywords))
        labels.append(category[i])
        
print(data[0])
print(labels[0])

(500, 3000)
['一代', '一样', '一线', '一致', '一般', '一角', '丁金华', '万元', '丈夫', '三星']
中华 主演 举行 互动 仪式 作为 儿童 全国 关注 出席 出演 加入 参与 周边 呼吁 地区 基金 大使 天下 妈妈 娱乐 媒体 学习 孩子 宣传 家庭 希望 帮助 幸福 开心 快乐 意义 感觉 成立 投入 支持 收获 故事 新浪 新片 方式 机会 正式 活动 演员 电影 登陆 知识 祝福 积极 组织 经历 结束 继续 舒淇 节日 获得 著名 表示 觉得 讲述 进行 院线 食品
娛樂


In [5]:
## 將資料分為訓練與測試 ##
data = np.array(data)
labels = np.array(labels)

#test_size測試集比例
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.1, random_state=2019) 

## 將文本轉為0/1矩陣, 詞出現為1 ##
count_vec = CountVectorizer(binary = True)
x_train = count_vec.fit_transform(x_train)  
x_test  = count_vec.transform(x_test)

print(x_train[0])
print(x_train.shape)

  (0, 7)	1
  (0, 22)	1
  (0, 40)	1
  (0, 73)	1
  (0, 176)	1
  (0, 201)	1
  (0, 202)	1
  (0, 232)	1
  (0, 233)	1
  (0, 237)	1
  (0, 261)	1
  (0, 272)	1
  (0, 317)	1
  (0, 364)	1
  (0, 365)	1
  (0, 481)	1
  (0, 521)	1
  (0, 559)	1
  (0, 564)	1
  (0, 594)	1
  (0, 626)	1
  (0, 629)	1
  (0, 704)	1
  (0, 752)	1
  (0, 931)	1
  :	:
  (0, 1748)	1
  (0, 1764)	1
  (0, 1802)	1
  (0, 1888)	1
  (0, 1903)	1
  (0, 2066)	1
  (0, 2162)	1
  (0, 2209)	1
  (0, 2212)	1
  (0, 2262)	1
  (0, 2383)	1
  (0, 2420)	1
  (0, 2448)	1
  (0, 2494)	1
  (0, 2504)	1
  (0, 2552)	1
  (0, 2587)	1
  (0, 2662)	1
  (0, 2688)	1
  (0, 2707)	1
  (0, 2718)	1
  (0, 2722)	1
  (0, 2772)	1
  (0, 2807)	1
  (0, 2947)	1
(450, 2982)


In [6]:
## start training and precision ##
clf=tree.DecisionTreeClassifier(max_depth=10, random_state=2020)
clf.fit(x_train, y_train)

DecisionTreeClassifier(max_depth=10, random_state=2020)

In [7]:
import os
os.environ["PATH"] += os.pathsep + 'C:/Users/leeys/Anaconda3/Library/bin/graphviz'

In [8]:
# draw tree  ##
import graphviz 
dot_data = tree.export_graphviz(clf, out_file=None, feature_names=count_vec.get_feature_names(),)
graph = graphviz.Source(dot_data) 

graph.format = 'svg'
graph.render(cwd+"/data/tree" ,view=True)

'C:\\Users\\leeys\\Desktop\\workspace\\文本分析/data\\tree.svg'

In [9]:
y_true = y_test
y_pred = clf.predict(x_test)

print(y_pred[:10])
print(y_true[:10])

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_true, y_pred, labels = ['娛樂','社會','財金','遊戲','體育']))

['娛樂' '遊戲' '娛樂' '財金' '娛樂' '體育' '社會' '財金' '社會' '社會']
['娛樂' '遊戲' '娛樂' '財金' '娛樂' '體育' '娛樂' '財金' '社會' '娛樂']
[[11  2  0  0  0]
 [ 0  8  0  0  0]
 [ 0  3 12  0  0]
 [ 0  0  0  4  0]
 [ 0  1  0  1  8]]


In [39]:
#precision and recall  
#precision, recall, thresholds = precision_recall_curve(y_test, clf.predict(x_test))  
    
print(classification_report(y_true, y_pred , target_names = ['娛樂','社會','財金','遊戲','體育']))
print("--------------------")
from sklearn.metrics import accuracy_score
print('準確率: %.2f' % accuracy_score(y_true, y_pred))
print('準確率: %.2f' % np.mean(y_true == y_pred))#預測值與真實值

              precision    recall  f1-score   support

          娛樂       1.00      0.85      0.92        13
          社會       0.62      1.00      0.76         8
          財金       1.00      0.87      0.93        15
          遊戲       0.80      1.00      0.89         4
          體育       1.00      0.80      0.89        10

    accuracy                           0.88        50
   macro avg       0.88      0.90      0.88        50
weighted avg       0.92      0.88      0.89        50

--------------------
準確率: 0.88
準確率: 0.88


In [40]:
y_true = y_train
y_pred = clf.predict(x_train) 

#precision and recall  
#precision, recall, thresholds = precision_recall_curve(y_test, clf.predict(x_test))  
    
print(classification_report(y_true, y_pred , target_names = ['娛樂','社會','財金','遊戲','體育']))
print("--------------------")
from sklearn.metrics import accuracy_score
print('準確率: %.2f' % accuracy_score(y_true, y_pred))
print('準確率: %.2f' % np.mean(y_true == y_pred))#預測值與真實值

              precision    recall  f1-score   support

          娛樂       0.99      1.00      0.99        87
          社會       0.92      1.00      0.96        92
          財金       1.00      1.00      1.00        85
          遊戲       1.00      0.98      0.99        96
          體育       1.00      0.92      0.96        90

    accuracy                           0.98       450
   macro avg       0.98      0.98      0.98       450
weighted avg       0.98      0.98      0.98       450

--------------------
準確率: 0.98
準確率: 0.98
